In [1]:
import pandas as pd
import numpy as np
import napari
import pytest

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.preprocessing.illumination_correction import IlluminationCorrection
import blimp.processing.quantify

05-Nov-24 21:05:38 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [2]:
viewer = napari.Viewer()

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:

_data_dir = Path("/Users/z3532965/src/blimp/tests/_data/")
intensity_image_2D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_intensity_image_TYX.tiff")
label_image_2D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_label_image_TYX.tiff")
intensity_image_3D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_intensity_image_TZYX.tiff")
label_image_3D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_label_image_TZYX.tiff")


``_measure_parent_object_label``

In [5]:
with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    blimp.processing.quantify._measure_parent_object_label(
        label_image = label_image_2D,
        measure_object_index = 2,
        parent_object_index = 0,
        timepoint = 0)

res = blimp.processing.quantify._measure_parent_object_label(
    label_image = label_image_2D,
    measure_object_index = 1,
    parent_object_index = 0,
    timepoint = 0)

res.to_csv(_data_dir / "resources" / "measure_parent_label_2D_results_t_0.csv")

[viewer.add_labels(label_image_2D.get_image_data("YX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


05-Nov-24 16:39:07 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.
05-Nov-24 16:39:07 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.


[<Labels layer 'Object1' at 0x334d5ab30>,
 <Labels layer 'Object2' at 0x334d68670>,
 <Labels layer 'Object3' at 0x345cef8b0>]

In [6]:
# check visually that this looks ok

# Convert DataFrame to dictionary for mapping
parent_labels_dict = dict(zip(res['label'], res['parent_label']))

# Replace values in the 2D array using np.vectorize with the dictionary
original_labels_array = label_image_2D.get_image_data("YX",C=1)
parent_labels_array = np.vectorize(parent_labels_dict.get)(original_labels_array, original_labels_array)
viewer.add_labels(parent_labels_array)


<Labels layer 'parent_labels_array' at 0x346168760>

3D

In [7]:
with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    blimp.processing.quantify._measure_parent_object_label(
        label_image = label_image_3D,
        measure_object_index = 2,
        parent_object_index = 0,
        timepoint = 0)

res = blimp.processing.quantify._measure_parent_object_label(
    label_image = label_image_3D,
    measure_object_index = 1,
    parent_object_index = 0,
    timepoint = 0)

res.to_csv(_data_dir / "datasets" / "synthetic_images" / "measure_parent_label_3D_results_t_0.csv")

[viewer.add_labels(label_image_3D.get_image_data("ZYX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


05-Nov-24 16:39:12 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.


05-Nov-24 16:39:14 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.


[<Labels layer 'Object1 [1]' at 0x3461a3eb0>,
 <Labels layer 'Object2 [1]' at 0x3658cec80>,
 <Labels layer 'Object3 [1]' at 0x3461a2380>]

In [8]:
# check visually that this looks ok

# Convert DataFrame to dictionary for mapping
parent_labels_dict = dict(zip(res['label'], res['parent_label']))

# Replace values in the 2D array using np.vectorize with the dictionary
original_labels_array = label_image_3D.get_image_data("ZYX",C=1)
parent_labels_array = np.vectorize(parent_labels_dict.get)(original_labels_array, original_labels_array)
viewer.add_labels(parent_labels_array)

<Labels layer 'parent_labels_array [1]' at 0x3464cb9d0>

``_quantify_single_timepoint_2D``

In [9]:

# no parent object
res_obj1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0)

res_obj1.Object1_intensity_max_Channel1.to_list() == [1000.,2000.,3000.,4000.]
res_obj1.Object1_area.to_list() == [100.0,81.0,900.0,961.0]

res_obj2 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0)

res_obj2.count().label==100
list(np.unique(res_obj2.Object2_area)) == [4.0]

with pytest.raises(AttributeError):
    # check no parent label when parent_label is none
    parents = res_obj2.parent_label

# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    res_obj3_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
        intensity_image = intensity_image_2D,
        label_image = label_image_2D,
        measure_object = 2,
        parent_object = 0,
        timepoint = 0)

# quantify object channel 1 relative to parent object channel 0
res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    parent_object = 0,
    timepoint = 0)

list(np.unique(res_obj2_parent1.parent_label_name)) == ["Object1"]

# check the number of objects in each parent object is correct
res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list() == [5,4,21,20]


05-Nov-24 16:39:18 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.
05-Nov-24 16:39:18 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.
05-Nov-24 16:39:18 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.


True

In [10]:
# test intensity_channels
res_obj2_intensity1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels="Channel1")

res_obj2_intensity1_list1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1"])

res_obj2_intensity1_list2_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1","Channel2"])

res_obj2_intensity1_list_int_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=[0,1])

with pytest.raises(AttributeError):
    res_obj2_intensity1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel3

list(np.unique(res_obj2_intensity1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel2)) == [0,5000.]


True

In [11]:
# test texture_channels

# calculate_textures = False overrides texture channels provided
res_obj1_texture1_str_input_texture_false = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=False,
    texture_channels="Channel1")

with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel1_Haralick-angular-second-moment-1']

# calculate_textures = True with texture channels not specified calculates all
res_obj1_texture_none_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True)

len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

# calculate_textures = True with texture channels specified calculates only those specified
res_obj1_texture1_str_input_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True,
    texture_channels="Channel1")

res_obj1_texture1_str_input_texture_true['Object1_Channel1_Haralick-angular-second-moment-1'].to_list() == [1.,1.,1.,1.]
with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel3_Haralick-angular-second-moment-1']


In [12]:
len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

True

In [13]:
# test no border objects
res_no_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

not any(res_no_border.Object1_is_border)

# test one border object (crop image to generate a border object)
res_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = AICSImage(
        intensity_image_2D.data[:,:,:,52:,52:],
        channel_names=intensity_image_2D.channel_names),
    label_image = AICSImage(
        label_image_2D.data[:,:,:,52:,52:],
        channel_names=label_image_2D.channel_names),
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

res_border.query("label==1").Object1_is_border[0]==True


True

``_quantify_single_timepoint_3D``

In [14]:
label_image_3D.channel_names
intensity_image_3D.channel_names

['Channel1', 'Channel2', 'Channel3']

In [17]:
# no parent object

res_obj1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    timepoint = 0)

res_obj1['Object1-3D-MIP_area'].to_list()==[10.**2, 9.**2, 30.**2, 46.**2]
res_obj1['Object1-3D_area'].to_list()==[10.**3, 9.**3, 30.**3, 46.**3]
res_obj1['Object1-3D-Middle_perimeter'].to_list()==[10.*4-4, 9.*4-4, 30.*4-4, 46.*4-4]

res_obj2 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0)

res_obj2.count().label==100
list(np.unique(res_obj2['Object2-3D_area'].to_list())) == [2.**3]

with pytest.raises(AttributeError):
    # check no parent label when parent_label is none
    parents = res_obj2.parent_label

# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image=intensity_image_3D,
    label_image=label_image_3D,
    measure_object=1,
    parent_object=0,
    timepoint=0
)

res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list()

05-Nov-24 16:43:34 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
05-Nov-24 16:43:34 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
05-Nov-24 16:43:40 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.
05-Nov-24 16:43:40 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
05-Nov-24 16:43:40 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
05-Nov-24 16:43:43 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.


[7, 5, 15, 13]

In [18]:
res_obj2.columns

Index(['label', 'Object2-3D_centroid_0', 'Object2-3D_centroid_1',
       'Object2-3D_centroid_2', 'Object2-3D_area', 'Object2-3D_area_convex',
       'Object2-3D_axis_major_length', 'Object2-3D_axis_minor_length',
       'Object2-3D_extent', 'Object2-3D_feret_diameter_max',
       'Object2-3D_solidity', 'Object2-3D_intensity_mean_Channel1',
       'Object2-3D_intensity_max_Channel1',
       'Object2-3D_intensity_min_Channel1', 'Object2-3D_intensity_sd_Channel1',
       'Object2-3D_intensity_median_Channel1',
       'Object2-3D_intensity_mean_Channel2',
       'Object2-3D_intensity_max_Channel2',
       'Object2-3D_intensity_min_Channel2', 'Object2-3D_intensity_sd_Channel2',
       'Object2-3D_intensity_median_Channel2',
       'Object2-3D_intensity_mean_Channel3',
       'Object2-3D_intensity_max_Channel3',
       'Object2-3D_intensity_min_Channel3', 'Object2-3D_intensity_sd_Channel3',
       'Object2-3D_intensity_median_Channel3', 'Object2-3D-MIP_area',
       'Object2-3D-MIP_area_con

In [ ]:
[viewer.add_labels(label_image_3D.get_image_data("ZYX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


[<Labels layer 'Object1 [2]' at 0x3453dc6d0>,
 <Labels layer 'Object2 [2]' at 0x3453df4c0>,
 <Labels layer 'Object3 [2]' at 0x36226f070>]

In [19]:
# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    res_obj3_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
        intensity_image = intensity_image_3D,
        label_image = label_image_3D,
        measure_object = 2,
        parent_object = 0,
        timepoint = 0)

# quantify object channel 1 relative to parent object channel 0
res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    parent_object = 0,
    timepoint = 0)

list(np.unique(res_obj2_parent1.parent_label_name)) == ["Object1"]

# check the number of objects in each parent object is correct
res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list() == [7, 5, 15, 13]

05-Nov-24 16:53:11 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
05-Nov-24 16:53:11 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
05-Nov-24 16:53:20 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.
05-Nov-24 16:53:21 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
05-Nov-24 16:53:21 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
05-Nov-24 16:53:29 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.
05-Nov-24 16:53:29 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
05-Nov-24 16:53:29 - bl

True

In [4]:
# test intensity_channels
res_obj2_intensity1_str_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels="Channel1")

res_obj2_intensity1_list1_str_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1"])

res_obj2_intensity1_list2_str_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1","Channel2"])

res_obj2_intensity1_list_int_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=[0,1])

with pytest.raises(AttributeError):
    res_obj2_intensity1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel3

list(np.unique(res_obj2_intensity1_str_input['Object2-3D_intensity_min_Channel1'].to_list())) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input['Object2-3D_intensity_min_Channel1'].to_list())) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input['Object2-3D_intensity_min_Channel2'].to_list())) == [5000.,6000.]


True

In [5]:
res_obj2_intensity1_list_int_input.columns

Index(['label', 'Object2-3D_centroid_0', 'Object2-3D_centroid_1',
       'Object2-3D_centroid_2', 'Object2-3D_area', 'Object2-3D_area_convex',
       'Object2-3D_axis_major_length', 'Object2-3D_axis_minor_length',
       'Object2-3D_extent', 'Object2-3D_feret_diameter_max',
       'Object2-3D_solidity', 'Object2-3D_intensity_mean_Channel2',
       'Object2-3D_intensity_max_Channel2',
       'Object2-3D_intensity_min_Channel2', 'Object2-3D_intensity_sd_Channel2',
       'Object2-3D_intensity_median_Channel2',
       'Object2-3D_intensity_mean_Channel1',
       'Object2-3D_intensity_max_Channel1',
       'Object2-3D_intensity_min_Channel1', 'Object2-3D_intensity_sd_Channel1',
       'Object2-3D_intensity_median_Channel1', 'Object2-3D-MIP_area',
       'Object2-3D-MIP_area_convex', 'Object2-3D-MIP_axis_major_length',
       'Object2-3D-MIP_axis_minor_length', 'Object2-3D-MIP_eccentricity',
       'Object2-3D-MIP_extent', 'Object2-3D-MIP_feret_diameter_max',
       'Object2-3D-MIP_solidity

In [ ]:
list(np.unique(res_obj2_intensity1_str_input.Object2_3D_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input.Object2_3D_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input.Object2_3D_intensity_min_Channel2)) == [5000.,6000.]

True

In [6]:
res_obj2_intensity1_str_input.columns

Index(['label', 'Object2-3D_centroid_0', 'Object2-3D_centroid_1',
       'Object2-3D_centroid_2', 'Object2-3D_area', 'Object2-3D_area_convex',
       'Object2-3D_axis_major_length', 'Object2-3D_axis_minor_length',
       'Object2-3D_extent', 'Object2-3D_feret_diameter_max',
       'Object2-3D_solidity', 'Object2-3D_intensity_mean_Channel1',
       'Object2-3D_intensity_max_Channel1',
       'Object2-3D_intensity_min_Channel1', 'Object2-3D_intensity_sd_Channel1',
       'Object2-3D_intensity_median_Channel1', 'Object2-3D-MIP_area',
       'Object2-3D-MIP_area_convex', 'Object2-3D-MIP_axis_major_length',
       'Object2-3D-MIP_axis_minor_length', 'Object2-3D-MIP_eccentricity',
       'Object2-3D-MIP_extent', 'Object2-3D-MIP_feret_diameter_max',
       'Object2-3D-MIP_solidity', 'Object2-3D-MIP_perimeter',
       'Object2-3D-MIP_perimeter_crofton', 'Object2-3D-MIP_euler_number',
       'Object2-3D-MIP_intensity_mean_Channel1',
       'Object2-3D-MIP_intensity_max_Channel1',
       'Object

In [ ]:
# test texture_channels

# calculate_textures = False overrides texture channels provided
res_obj1_texture1_str_input_texture_false = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=False,
    texture_channels="Channel1")

with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel1_Haralick-angular-second-moment-1']

# calculate_textures = True with texture channels not specified calculates all
res_obj1_texture_none_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True)

len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

# calculate_textures = True with texture channels specified calculates only those specified
res_obj1_texture1_str_input_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True,
    texture_channels="Channel1")

res_obj1_texture1_str_input_texture_true['Object1_Channel1_Haralick-angular-second-moment-1'].to_list() == [1.,1.,1.,1.]
with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel3_Haralick-angular-second-moment-1']


In [ ]:
len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

True

In [7]:
# test no border objects
res_no_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

not any(res_no_border.Object1_is_border)

# test one border object (crop image to generate a border object)
res_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = AICSImage(
        intensity_image_2D.data[:,:,:,52:,52:],
        channel_names=intensity_image_2D.channel_names),
    label_image = AICSImage(
        label_image_2D.data[:,:,:,52:,52:],
        channel_names=label_image_2D.channel_names),
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

res_border.query("label==1").Object1_is_border


0    True
Name: Object1_is_border, dtype: bool

In [8]:
res_no_border = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

not any(res_no_border['Object1-3D_is_border'].to_list())

True

In [9]:
res_no_border

,label,Object1-3D_centroid_0,Object1-3D_centroid_1,Object1-3D_centroid_2,Object1-3D_area,Object1-3D_area_convex,Object1-3D_axis_major_length,Object1-3D_axis_minor_length,Object1-3D_extent,Object1-3D_feret_diameter_max,...,Object1-3D-Middle_perimeter_crofton,Object1-3D-Middle_euler_number,Object1-3D-Middle_intensity_mean_Channel1,Object1-3D-Middle_intensity_max_Channel1,Object1-3D-Middle_intensity_min_Channel1,Object1-3D-Middle_intensity_sd_Channel1,Object1-3D-Middle_intensity_median_Channel1,Object1-3D_is_border,Object1-3D_is_border_XY,TimepointID
0,1,9.5,9.5,9.5,1000.0,1000.0,12.845233,12.845233,1.0,16.186414,...,36.811657,1,1000.0,1000.0,1000.0,0.0,1000.0,False,False,1
1,2,24.0,24.0,24.0,729.0,729.0,11.547005,11.547005,1.0,14.456832,...,33.019419,1,2000.0,2000.0,2000.0,0.0,2000.0,False,False,1
2,3,64.5,64.5,64.5,27000.0,27000.0,38.708311,38.708311,1.0,50.813384,...,112.656413,1,3000.0,3000.0,3000.0,0.0,3000.0,False,False,1
3,4,122.5,27.5,27.5,97336.0,97336.0,59.371710,59.371710,1.0,78.523882,...,173.332218,1,4000.0,4000.0,4000.0,0.0,4000.0,False,False,1


In [10]:
viewer3D = napari.Viewer()


05-Nov-24 20:01:31 - OpenGL.acceleratesupport - INFO     - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [11]:
[viewer3D.add_labels(label_image_3D.get_image_data("ZYX",C=i),name=label_image_3D.channel_names[i]) for i in range(label_image_3D.dims.C)]
[viewer3D.add_image(intensity_image_3D.get_image_data("ZYX",C=i),name=intensity_image_3D.channel_names[i]) for i in range(intensity_image_3D.dims.C)]

[<Image layer 'Channel1' at 0x36c17a9b0>,
 <Image layer 'Channel2' at 0x37b50b8b0>,
 <Image layer 'Channel3' at 0x37b77f1c0>]

In [ ]:
# test one border object (crop image to generate a border object)
res_border_all = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = AICSImage(
        intensity_image_3D.data[:,:,:,8:,8:],
        channel_names=intensity_image_3D.channel_names,
        physical_pixel_sizes=intensity_image_3D.physical_pixel_sizes),
    label_image = AICSImage(
        label_image_3D.data[:,:,:,8:,8:],
        channel_names=label_image_3D.channel_names,
        physical_pixel_sizes=label_image_3D.physical_pixel_sizes),
    measure_object = 0,
    timepoint = 0,
    intensity_channels=0)

res_border_all.query("label==1")['Object1-3D_is_border']
res_border_all.query("label==1")['Object1-3D_is_border_XY']

res_border_Z_only = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = AICSImage(
        intensity_image_3D.data[:,:,8:,:,:],
        channel_names=intensity_image_3D.channel_names,
        physical_pixel_sizes=intensity_image_3D.physical_pixel_sizes),
    label_image = AICSImage(
        label_image_3D.data[:,:,8:,:,:],
        channel_names=label_image_3D.channel_names,
        physical_pixel_sizes=label_image_3D.physical_pixel_sizes),
    measure_object = 0,
    timepoint = 0,
    intensity_channels=0)

res_border_Z_only.query("label==1")['Object1-3D_is_border']
res_border_Z_only.query("label==1")['Object1-3D_is_border_XY']


0    True
Name: Object1-3D_is_border, dtype: bool

In [45]:
np.unique(intensity_image_3D.data[:,2,:,52:,52:])

array([   0, 7000, 8000], dtype=uint16)

In [29]:
res_no_border.columns


Index(['label', 'Object1_3D_centroid_0', 'Object1_3D_centroid_1',
       'Object1_3D_centroid_2', 'Object1_3D_area', 'Object1_3D_area_convex',
       'Object1_3D_axis_major_length', 'Object1_3D_axis_minor_length',
       'Object1_3D_extent', 'Object1_3D_feret_diameter_max',
       'Object1_3D_solidity', 'Object1_3D_intensity_mean_Channel1',
       'Object1_3D_intensity_max_Channel1',
       'Object1_3D_intensity_min_Channel1', 'Object1_3D_intensity_sd_Channel1',
       'Object1_3D_intensity_median_Channel1', 'Object1-3D-MIP_area',
       'Object1-3D-MIP_area_convex', 'Object1-3D-MIP_axis_major_length',
       'Object1-3D-MIP_axis_minor_length', 'Object1-3D-MIP_eccentricity',
       'Object1-3D-MIP_extent', 'Object1-3D-MIP_feret_diameter_max',
       'Object1-3D-MIP_solidity', 'Object1-3D-MIP_perimeter',
       'Object1-3D-MIP_perimeter_crofton', 'Object1-3D-MIP_euler_number',
       'Object1-3D-MIP_intensity_mean_Channel1',
       'Object1-3D-MIP_intensity_max_Channel1',
       'Object

In [4]:
res = blimp.processing.quantify.quantify(
    intensity_image=intensity_image_2D,
    label_image=label_image_2D,
)

05-Nov-24 20:31:50 - blimp.utils - INFO     - Channel names unique: ['Object1', 'Object2', 'Object3']
05-Nov-24 20:31:50 - blimp.utils - INFO     - Channel names unique: ['Channel1', 'Channel2', 'Channel3']
05-Nov-24 20:31:50 - blimp.processing.quantify - INFO     - ``measure_objects`` =  ['Object1', 'Object2', 'Object3']
05-Nov-24 20:31:50 - blimp.processing.quantify - INFO     - ``texture_objects`` =  ['Object1', 'Object2', 'Object3']
05-Nov-24 20:31:50 - blimp.processing.quantify - INFO     - ``parent_object`` =  None
05-Nov-24 20:31:50 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.
05-Nov-24 20:31:50 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.
05-Nov-24 20:31:53 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.


In [38]:
res = blimp.processing.quantify.quantify(
    intensity_image=intensity_image_2D,
    label_image=label_image_2D,
    measure_objects=[0, 1],
    parent_object=0,
    aggregate=True,
    timepoint=0
    )

05-Nov-24 21:21:58 - blimp.utils - INFO     - Channel names unique: ['Object1', 'Object2', 'Object3']
05-Nov-24 21:21:58 - blimp.utils - INFO     - Channel names unique: ['Channel1', 'Channel2', 'Channel3']
05-Nov-24 21:21:58 - blimp.processing.quantify - INFO     - ``measure_objects`` =  ['Object1', 'Object2']
05-Nov-24 21:21:58 - blimp.processing.quantify - INFO     - ``texture_objects`` =  ['Object1', 'Object2', 'Object3']
05-Nov-24 21:21:58 - blimp.processing.quantify - INFO     - ``parent_object`` =  Object1
05-Nov-24 21:21:58 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.
05-Nov-24 21:21:58 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.
05-Nov-24 21:21:58 - blimp.processing.quantify - INFO     - ``intensity_image`` is 2D. Quantifying 2D features only.
05-Nov-24 21:21:59 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.


In [45]:
res.Object2_intensity_min_Channel2_max

0       0.0
1    5000.0
2    5000.0
3       0.0
Name: Object2_intensity_min_Channel2_max, dtype: float64

In [ ]:
features_non_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=False)

In [ ]:
len(features_non_aggregated)

In [ ]:
features_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=True)

In [ ]:
features_aggregated.columns